In [2]:
from langchain.chat_models import ChatOpenAI
from pyspark_ai import SparkAI

# If 'gpt-4' is unavailable, use 'gpt-3.5-turbo' (might lower output quality)
# llm = ChatOpenAI(model_name='gpt-3.5-turbo', temperature=0)
# spark_ai = SparkAI(llm=llm, verbose=True)

# Initialize SparkAI with the ChatOpenAI model

spark_ai = SparkAI()
# spark_ai = SparkAI()
# Activate partial functions for Spark DataFrame
spark_ai.activate()

INFO: The pyspark.sql.connect.dataframe module could not be imported. This might be due to your PySpark version being below 3.4.


In [30]:
# auto_df = spark_ai.create_df("https://github.com/airbytehq/airbyte/pulls/")
# auto_df = spark_ai.create_df("top ranked GitHub repositories of open source")

crypto_df = spark_ai.create_df('What are the cryptocurrency price for today', ['cryptocurrency_name','price','change_24h','volume_24h','market_cap'])
spark_ai.commit()
crypto_df.show()

INFO: Parsing URL: https://crypto.com/price

INFO: SQL query for the ingestion:
CREATE OR REPLACE TEMP VIEW spark_ai_temp_view_792142 AS 
SELECT * FROM VALUES
('Bitcoin', 43646.90, -0.35, 15.44, 854.6),
('Ethereum', 2287.99, -0.59, 8.98, 274.98),
('Tether', 1.00, -0.03, 46.19, 91.38),
('Solana', 115.40, 15.67, 7.12, 49.33),
('BNB', 269.23, -0.58, 0.79327, 40.84),
('XRP', 0.6215, 0.29, 0.99526, 33.6),
('USD Coin', 1.00, -0.03, 4.05, 25.09),
('Cardano', 0.6159, 1.87, 0.7222, 21.77),
('Avalanche', 48.58, 9.18, 1.86, 17.75),
('Dogecoin', 0.09379, 0.74, 0.50019, 13.35)
AS v1(cryptocurrency_name, price, change_24h, volume_24h, market_cap)

INFO: Storing data into temp view: spark_ai_temp_view_792142

+-------------------+-----------+----------+----------+----------+
|cryptocurrency_name|      price|change_24h|volume_24h|market_cap|
+-------------------+-----------+----------+----------+----------+
|            Bitcoin|43646.90000|     -0.35|  15.44000|    854.60|
|           Ethereum| 2287.9

In [52]:
github_trending = spark_ai.create_df("github trending repositories",['repo_name','owner','created_at'])
spark_ai.commit()
github_trending.show(truncate=False)


INFO: Parsing URL: https://github.com/trending

INFO: SQL query for the ingestion:
CREATE OR REPLACE TEMP VIEW spark_ai_temp_view_263671 AS 
SELECT * FROM VALUES
('gpt-pilot', 'Pythagora-io', '2022-01-01'),
('AppAgent', 'mnotgod96', '2022-01-02'),
('StreamDiffusion', 'cumulo-autumn', '2022-01-03') 
AS v1(repo_name, owner, created_at)

INFO: Storing data into temp view: spark_ai_temp_view_263671

+---------------+-------------+----------+
|repo_name      |owner        |created_at|
+---------------+-------------+----------+
|gpt-pilot      |Pythagora-io |2022-01-01|
|AppAgent       |mnotgod96    |2022-01-02|
|StreamDiffusion|cumulo-autumn|2022-01-03|
+---------------+-------------+----------+



In [54]:
all_github_trending = spark_ai.create_df("all github trending repositories",['repo_name','language','stars'])
spark_ai.commit()
all_github_trending.show(truncate=False)
all_github_trending.count()

INFO: Parsing URL: https://github.com/trending

INFO: SQL query for the ingestion:
CREATE OR REPLACE TEMP VIEW spark_ai_temp_view_263671 AS SELECT * FROM VALUES
('Pythagora-io / gpt-pilot', 'Python', 16761),
('mnotgod96 / AppAgent', 'Python', 725),
('cumulo-autumn / StreamDiffusion', 'Python', 2429),
('ton-community / tsc5', 'TypeScript', 35),
('mckaywrigley / chatbot-ui', 'TypeScript', 19598),
('Anuken / Mindustry', 'Java', 20057),
('NationalSecurityAgency / ghidra', 'Java', 44841),
('beeper / imessage', 'Go', 683),
('labring / FastGPT', 'TypeScript', 7486),
('heyman / heynote', 'JavaScript', 1002),
('AmruthPillai / Reactive-Resume', 'TypeScript', 15134),
('papers-we-love / papers-we-love', 'Shell', 78920),
('jwasham / coding-interview-university', NULL, 273884),
('guoqincode / AnimateAnyone-unofficial', 'Python', 676),
('lllyasviel / Fooocus', 'Python', 26330),
('SillyTavern / SillyTavern', NULL, NULL)
AS v1(repo_name, language, stars)

INFO: Storing data into temp view: spark_ai_tem

16

In [58]:
all_github_trending.ai.transform("What are the higher and second highest ranked stars repository").show(truncate=False)


INFO: Creating temp view for the transform:
df.createOrReplaceTempView("spark_ai_temp_view_2139859579")

INFO: Using cached result for the transform:
INFO: SELECT repo_name, stars FROM spark_ai_temp_view ORDER BY stars DESC LIMIT 2

+-------------------------------------+------+
|repo_name                            |stars |
+-------------------------------------+------+
|jwasham / coding-interview-university|273884|
|papers-we-love / papers-we-love      |78920 |
+-------------------------------------+------+



In [57]:
all_github_trending.ai.transform("What are the higher and second highest ranked stars repository in each language").show(truncate=False)

INFO: Creating temp view for the transform:
df.createOrReplaceTempView("spark_ai_temp_view_2139859579")

INFO: Using cached result for the transform:
INFO: SELECT language, repo_name, stars FROM (SELECT language, repo_name, stars, ROW_NUMBER() OVER (PARTITION BY language ORDER BY stars DESC) as rank FROM spark_ai_temp_view) WHERE rank <= 2

+----------+-------------------------------------+------+
|language  |repo_name                            |stars |
+----------+-------------------------------------+------+
|NULL      |jwasham / coding-interview-university|273884|
|NULL      |SillyTavern / SillyTavern            |NULL  |
|Go        |beeper / imessage                    |683   |
|Java      |NationalSecurityAgency / ghidra      |44841 |
|Java      |Anuken / Mindustry                   |20057 |
|JavaScript|heyman / heynote                     |1002  |
|Python    |lllyasviel / Fooocus                 |26330 |
|Python    |Pythagora-io / gpt-pilot             |16761 |
|Shell     |papers-

In [37]:

airbyte_pulls = spark_ai.create_df("https://github.com/airbytehq/airbyte/pulls/",['pr_title','pr_created_at','pr_created_by', 'pr_status','pr_label'])
spark_ai.commit
airbyte_pulls.show(truncate=False)



INFO: Parsing URL: https://github.com/airbytehq/airbyte/pulls/

INFO: SQL query for the ingestion:
CREATE OR REPLACE TEMP VIEW spark_ai_temp_view_544707 AS 
SELECT * FROM VALUES
('Fix CDCPOstgresSourceFullCertificateSslAcceptanceTest', '2023-12-24', 'stephane-airbyte', 'Open', 'CDK Connector Development Kit'),
('Use base64 to encode mssql binary field', '2023-12-22', 'xiaohansong', 'Open', 'CDK Connector Development Kit')
AS v1(pr_title, pr_created_at, pr_created_by, pr_status, pr_label)

INFO: Storing data into temp view: spark_ai_temp_view_544707

+-----------------------------------------------------+-------------+----------------+---------+-----------------------------+
|pr_title                                             |pr_created_at|pr_created_by   |pr_status|pr_label                     |
+-----------------------------------------------------+-------------+----------------+---------+-----------------------------+
|Fix CDCPOstgresSourceFullCertificateSslAcceptanceTest|2023-12-